# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages
import chardet as chardet
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root, '*'))
    #print(file_path_list)

/home/sfrozy/PycharmProjects/Udacity-DEND/project_2_data_modeling_with_apache_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []

# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file
    with open(f, 'r', encoding='utf8', newline='') as csvfile:
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        next(csvreader)

        # extracting each data row one by one and append it
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

        # uncomment the code below if you would like to get total number of rows
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding='utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
                     'level', 'location', 'sessionId', 'song', 'userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding='utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                    )

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

#### Header Mapper

In [9]:
file = 'event_datafile_new.csv'

header_dict = {}

# read header from first line of csv file
with open(file, 'r', encoding='utf8') as f:
    header = f.readline()
    print(type(header))
    header = header.replace('"', '').replace('\n', '').split(',')

     # map header to a dictionary with index
    for i, h in enumerate(header):
        header_dict[h] = i

header_dict

<class 'str'>


{'artist': 0,
 'firstName': 1,
 'gender': 2,
 'itemInSession': 3,
 'lastName': 4,
 'length': 5,
 'level': 6,
 'location': 7,
 'sessionId': 8,
 'song': 9,
 'userId': 10}

#### Guessing encoding

In [10]:
import chardet

file = 'event_datafile_new.csv'

# look at the first ten thousand bytes to guess the character encoding
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

{'encoding': 'utf-8', 'confidence': 0.938125, 'language': ''}


In [11]:
import chardet

file = 'event_data/2018-11-02-events.csv'

# look at the first ten thousand bytes to guess the character encoding
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [12]:
table_name_1 = 'song_by_session'

In [13]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

select_query_1 = f"SELECT artist, song, length FROM {table_name_1} WHERE sessionId = 338 AND itemInSession = 4"

Based on the query, I will include the `session_id` as the primary key, and the `item_in_session` as the clustering key.

In [14]:
# create table

create_table_query = f"""CREATE TABLE IF NOT EXISTS
                            {table_name_1} (sessionId int, itemInSession int, artist text, song text, length float,
                            PRIMARY KEY (sessionId, itemInSession))"""

session.execute(create_table_query)

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = f"INSERT INTO {table_name_1} (sessionId, itemInSession, artist, song, length) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [16]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

rows = session.execute(select_query_1)

for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [17]:
table_name_2 = 'song_by_user'

In [18]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

select_query_2 = f"SELECT artist, song, firstName, lastName FROM {table_name_2} WHERE userId = 10 AND sessionId = 182"

Based on the query, I will include the `sessionId` as the primary key, and the `itemInSession` and `userId` as the clustering key. Because it need to be sorted in `userId` and `itemInSession` order, I will use both of this keys as the clustering key.

In [19]:
# create table
# session.execute("DROP TABLE song_data_2")

create_table_query = f"""CREATE TABLE IF NOT EXISTS
                        {table_name_2} (sessionId int, itemInSession int, artist text, song text, userId int, firstName text, lastName text,
                        PRIMARY KEY (sessionId, userId, itemInSession))"""


session.execute(create_table_query)

In [20]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = f"INSERT INTO {table_name_2} (sessionId, itemInSession, artist, song, userId, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[header_dict.get('sessionId')]),
                                int(line[header_dict.get('itemInSession')]),
                                line[header_dict.get('artist')],
                                line[header_dict.get('song')],
                                int(line[header_dict.get('userId')]),
                                line[header_dict.get('firstName')],
                                line[header_dict.get('lastName')]))

In [21]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute(select_query_2)

for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [22]:
table_name_3 = 'user_by_song'

In [23]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

select_query_3 = f"SELECT firstName, lastName FROM {table_name_3} WHERE song = 'All Hands Against His Own'"

Because we are looking user which listened to the song 'All Hands Against His Own', we will use the `song` as the primary key. and using the `userId` as the clustering key.

In [24]:
# create table
# session.execute("DROP TABLE song_data_3")

create_table_query = f"""CREATE TABLE IF NOT EXISTS
                    {table_name_3} (song text, userId int, firstName text, lastName text,
                    PRIMARY KEY (song, userId))"""


session.execute(create_table_query)

In [25]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = f"INSERT INTO {table_name_3} (song, userId, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[header_dict.get('song')],
                                int(line[header_dict.get('userId')]),
                                line[header_dict.get('firstName')],
                                line[header_dict.get('lastName')]))

In [26]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute(select_query_3)

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [27]:
## TO-DO: Drop the table before closing out the sessions
for table in [table_name_1, table_name_2, table_name_3]:
    session.execute(f"DROP TABLE {table}")

### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()